# Evaluation notebook for visualization purposes

In [1]:
# import torch.nn.functional as F
import torch
# from faceEmbeddingModel import faceEmbeddingModel
# from refFaceEmbeddingModel import refFaceEmbeddingModel
# from reg_database import RegistrationDatabase
# from prep import load_and_transform_img
# from pipeline_evaluation import PipelineEvaluation
import sys
import numpy as np
from LFWEvaluationDatasetCropped import LFWEvaluationDatasetCropped

In [2]:
# dataset_path = "./evaluation_data/lfw_crop"
# eval_log_path = "./evaluation_results/ref_model_with_augmentation.txt"
# #face_detection_model = DeepFace
# ref_face_embedding_model = refFaceEmbeddingModel()
# face_embedding_model = faceEmbeddingModel().eval()
# registration_database = RegistrationDatabase()

# pipeline_evaluation = PipelineEvaluation(dataset_path, eval_log_path,
#                                           ref_face_embedding_model, registration_database)

# pipeline_evaluation.run()
#pipeline_evaluation.plot_results()

In [3]:
dataset_path = "./evaluation_data/lfw_crop"
eval_dataset = LFWEvaluationDatasetCropped(dataset_path)

In [4]:
# ---- SHUFFLE DATASET RANDOMLY --------
# divide dataset size by twenty 10, so that processing is faster
subset_size = 1
n_samples = int(eval_dataset.__len__()/subset_size)

# Shuffle indices with np.random.permutation, also fix seed if you want reproducability
shuffled_indices = np.random.RandomState(seed=42).permutation(n_samples)

# select shuffled set from original dataset
eval_dataset_shuffled = torch.utils.data.Subset(eval_dataset, indices=shuffled_indices)   

# ---- CREATE DATALOADER --------
batch_size = 1
eval_loader = torch.utils.data.DataLoader(dataset=eval_dataset_shuffled,
                                            batch_size=batch_size,
                                            num_workers=0,
                                            shuffle=False, 
                                            sampler=None,
                                            collate_fn=None)

In [5]:
print(len(eval_loader))

5409


In [6]:
len(eval_dataset.image_filenames)

5409

In [7]:
len(eval_dataset.labels)

5409

In [8]:
eval_dataset.image_filenames

['./evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0001.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0002.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0003.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0004.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0005.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0006.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0007.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0008.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0009.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0010.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0011.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0012.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0013.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdullah_Gul_0014.jpg',
 './evaluation_data/lfw_crop\\Abdullah_Gul\\Abdu

In [9]:
eval_dataset.labels

['Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Abdullah_Gul',
 'Adrien_Brody',
 'Adrien_Brody',
 'Adrien_Brody',
 'Adrien_Brody',
 'Adrien_Brody',
 'Adrien_Brody',
 'Adrien_Brody',
 'Adrien_Brody',
 'Adrien_Brody',
 'Adrien_Brody',
 'Adrien_Brody',
 'Adrien_Brody',
 'Albert_Costa',
 'Albert_Costa',
 'Albert_Costa',
 'Albert_Costa',
 'Albert_Costa',
 'Albert_Costa',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_Toledo',
 'Alejandro_To

In [10]:
for label, image in eval_loader:
    print(label)
    print(image)
    sys.exit()

('Tony_Blair',)
tensor([[[[0.2471, 0.2431, 0.2392,  ..., 0.3765, 0.3647, 0.3608],
          [0.2549, 0.2510, 0.2510,  ..., 0.3804, 0.3725, 0.3647],
          [0.2588, 0.2627, 0.2627,  ..., 0.3843, 0.3804, 0.3765],
          ...,
          [0.3608, 0.3451, 0.3255,  ..., 0.9647, 0.9608, 0.9608],
          [0.3569, 0.3451, 0.3255,  ..., 0.9647, 0.9647, 0.9647],
          [0.3569, 0.3451, 0.3255,  ..., 0.9647, 0.9686, 0.9686]],

         [[0.2824, 0.2784, 0.2745,  ..., 0.3569, 0.3451, 0.3412],
          [0.2902, 0.2863, 0.2863,  ..., 0.3608, 0.3529, 0.3451],
          [0.2941, 0.2980, 0.2980,  ..., 0.3647, 0.3608, 0.3569],
          ...,
          [0.4353, 0.4196, 0.4000,  ..., 0.9686, 0.9686, 0.9686],
          [0.4314, 0.4196, 0.4000,  ..., 0.9725, 0.9725, 0.9725],
          [0.4314, 0.4196, 0.4000,  ..., 0.9725, 0.9765, 0.9765]],

         [[0.2157, 0.2118, 0.2078,  ..., 0.2824, 0.2706, 0.2667],
          [0.2235, 0.2196, 0.2196,  ..., 0.2863, 0.2784, 0.2706],
          [0.2196, 0.2235,

SystemExit: 

C:\Users\Marco\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
